#### THIS SCRIPT WILL ASSUME ALL CHANGES TO SETS HAVE BEEN COMPLETED ALREADY (ALLOCATION OF EXPERIMENTS, DETACHING FROM PUBLICATIONS...).
#### THIS SCRIPT WILL CHANGE THE STATUS OF THE OLD SET, AND ASSOCIATE IT WITH THE NEW ONE FOR REDIRECT AND ADD STATIC SECTIONS ABOUT REPLACEMENTS.

In [ ]:
### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
from dcicutils import ff_utils
from functions.notebook_functions import *
import json

import time

# set to replace the old one
new_set = '4DNESNGESP28'
# set to be replaced
old_set = '46db06ad-b399-4cf4-9acc-07b3e25ef132'
# reason for replacement
reason = 'new biological replicates were added'
# will perform patches/posts if set to true
action = False

# get key from keypairs.json
my_key = get_key('koray_data')
schema_name = get_schema_names(my_key) 
print('WORKING ON', my_key['server'], '\n')

old_set_info = ff_utils.get_metadata(old_set, my_key)
new_set_info = ff_utils.get_metadata(new_set, my_key)
old_acc = old_set_info['accession']
new_acc = new_set_info['accession']
old_status = old_set_info['status']
new_status = new_set_info['status']

if old_status == 'in review by lab':
    old_status = 'draft'
if new_status == 'in review by lab':
    new_status = 'draft'

# add headers to old and new set
old_alias = "static_header:replaced_item_{}_by_{}".format(old_acc, new_acc)
old_header = {
  "body": "This experiment set was replaced by [{0}](https://data.4dnucleome.org/experiment-set-replicates/{0}/) because {1}.".format(new_acc, reason),
  "award": old_set_info['award']['uuid'],
  "lab": old_set_info['lab']['uuid'],            
  "name": "static-header.replaced_item_{}".format(old_acc),
  "section_type": "Item Page Header",
  "options": {"title_icon": "info", "default_open": True, "filetype": "md", "collapsible": False},
  "title": "Note: Replaced Item - {}".format(old_acc),
  "status": old_status,
  "aliases": [old_alias]
}
new_alias = "static_header:replacing_item_{}_old_{}".format(new_acc, old_acc)
new_header = {
  "body": "This experiment set supercedes [{0}](https://data.4dnucleome.org/experiment-set-replicates/{1}/) because {2}.".format(old_acc, old_set_info['uuid'], reason),
  "award": new_set_info['award']['uuid'],
  "lab": new_set_info['lab']['uuid'],
  "name": "static-header.replacing_item_{}".format(new_acc),
  "section_type": "Item Page Header",
  "options": {"title_icon": "info", "default_open": True, "filetype": "md", "collapsible": False},
  "title": "Note: Replaced Item - {}".format(new_acc),
  "status": new_status,
  "aliases": [new_alias]
}
print('ADDING HEADER TO THE OLD SET')
print(old_header)
print('ADDING HEADER TO THE NEW SET')
print(new_header)

if action:
    # post the static sections
    try:
        old_h_resp = ff_utils.post_metadata(old_header, 'StaticSection', my_key)['@graph']
    except:
        print('old header already in system')
        old_h_resp = ff_utils.get_metadata(old_alias, my_key)
        
    try:
        new_h_resp = ff_utils.post_metadata(new_header, 'StaticSection', my_key)['@graph']
    except:
        print('new header already in system')
        new_h_resp = ff_utils.get_metadata(new_alias, my_key) 
    
    #see if existing headers
    old_header_list = []
    new_header_list = []
    if old_set_info.get('static_headers'):
        old_header_list = [i['uuid'] for i in old_set_info['static_headers']]
    if new_set_info.get('static_headers'):
        new_header_list = [i['uuid'] for i in new_set_info['static_headers']]
    # add new ones to the list
    if old_h_resp['uuid'] in old_header_list:
        pass
    else:
        old_header_list.append(old_h_resp['uuid'])
    if new_h_resp['uuid'] in new_header_list:
        pass
    else:
        new_header_list.append(new_h_resp['uuid'])
    
    # set the status of old set to replaced
    ff_utils.patch_metadata({'status':'replaced', 'static_headers': old_header_list},
                            obj_id=old_set_info['uuid'], key=my_key)
    # wait for indexing to take place
    # you might need to repeat this last piece separately if indexing does not catch up
    # new status needs to be indexed for alternate accession to be patched
    time.sleep(60)
    # set the alternate accession on the new set to the old one
    alt_ac = []
    if new_set_info.get('alternate_accessions'):
        alt_ac = new_set_info['alternate_accessions']
    alt_ac.append(old_acc)
    ff_utils.patch_metadata({'alternate_accessions':alt_ac, 'static_headers': new_header_list},
                            obj_id=new_set_info['uuid'], key=my_key)

print('DONE')
print('CHECK THE OLD SET', 'https://data.4dnucleome.org/experiment-set-replicates/{}/'.format(old_set_info['uuid']))
print('CHECK THE NEW SET', 'https://data.4dnucleome.org/experiment-set-replicates/{}/'.format(new_acc))